In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
sample_set = pd.read_table('../test_set_2/ts2_input.txt', names = ['Test'])

In [3]:
sample_set.info

<bound method DataFrame.info of                                                    Test
0                                                    50
1                                                    86
2     0010000010010010001011001110100101000000000000...
3     0111001010110010111000100110110101010000001110...
4     0110000000110011011001001101110101000001000100...
...                                                 ...
4997  0011110100110010010101011111011101100011001011...
4998  1101111101110110110111111111011101101101111001...
4999  1111111111111111110101111111111111111111101101...
5000  0011010000110011110111010101111001101001101001...
5001  0101110001101101110111011011011111110011111001...

[5002 rows x 1 columns]>

In [4]:
sample_set.head()

,Test
0,50
1,86
2,0010000010010010001011001110100101000000000000...
3,0111001010110010111000100110110101010000001110...
4,0110000000110011011001001101110101000001000100...


In [5]:
# Prvý riadok vstupu udáva počet testovacích prípadov T. 
T = sample_set.Test[0]

In [6]:
# Druhý riadok vstupu udáva percento testovacích prípadov P, na ktoré musíte správne odpovedať, 
# aby sa vaše riešenie považovalo za správne.
P = sample_set.Test[1]

In [7]:
n = 2
test_cases = sample_set.drop(index = sample_set.index[:n])

In [8]:
test_cases.info

<bound method DataFrame.info of                                                    Test
2     0010000010010010001011001110100101000000000000...
3     0111001010110010111000100110110101010000001110...
4     0110000000110011011001001101110101000001000100...
5     0010001010111010011011001000000100000000001100...
6     0111011100110110011010101110110101110100011100...
...                                                 ...
4997  0011110100110010010101011111011101100011001011...
4998  1101111101110110110111111111011101101101111001...
4999  1111111111111111110101111111111111111111101101...
5000  0011010000110011110111010101111001101001101001...
5001  0101110001101101110111011011011111110011111001...

[5000 rows x 1 columns]>

In [9]:
test_cases.head()

,Test
2,0010000010010010001011001110100101000000000000...
3,0111001010110010111000100110110101010000001110...
4,0110000000110011011001001101110101000001000100...
5,0010001010111010011011001000000100000000001100...
6,0111011100110110011010101110110101110100011100...


In [10]:
len(test_cases['Test'])

5000

In [11]:
def split(word):
    return [char for char in word]

In [12]:
# Keď sa hráč i pokúsi odpovedať na otázku j, pravdepodobnosť, 
# že na ňu odpovie správne, je f( Si − Qj ), kde f je sigmoidná funkcia:
import numpy as np

def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [13]:
def fair_probability(si, qj):
    return sigmoid(si - qj)

In [14]:
## Hráč i má úroveň zručností Si a otázka j má úroveň obtiažnosti Qj.
# každá obtiažnosť otázky sa vyberá jednotne a náhodne z rozsahu [−3.00, 3.00] 
decision_array = [-3.00, 3.00]

In [15]:
# a nezávisle od všetkých ostatných možností. 
Si = np.random.uniform(decision_array[0], decision_array[1])
Qj = np.random.uniform(decision_array[0], decision_array[1])
Si, Qj

(-2.749427080349873, 1.4356119684178283)

In [16]:
fair_probability(Si, Qj)

0.01499338801699691

In [17]:
unfair_probability = 0.5

In [18]:
# reset index
test_cases.reset_index(inplace=True)

In [19]:
 len(test_cases['Test'])/ 50

100.0

In [20]:
## Chunking
n = int(len(test_cases['Test']) / int(T))  #chunk row size
list_df = [test_cases[i:i+n] for i in range(0, test_cases.shape[0],n)]
len(list_df)

50

In [28]:
# %%time

# solutions = []
# for df in list_df:
#     row_lengts = 10000
#     Qjs = []
#     for i in range(row_lengts):
#         Qjs.append(np.random.uniform(decision_array[0], decision_array[1]))

#     diff_vals = []
#     for index, row in df.iterrows():
#         np_array = np.array(split(row['Test'])).astype('int32')
#         const_of_diff = 0
#         Si = np.random.uniform(decision_array[0], decision_array[1])

#         for i in range(len(np_array)):
#             proba = fair_probability(Si, Qjs[i])
#             response = 0
#             if proba > 0.5:
#                 response = 1

#             # print('index,', index, 'Si', Si, 'Qj', Qjs[i], 'probability', proba, 'our_prediction', response, 'real_val', np_array[i])
#             if np_array[i] != response:
#                 const_of_diff = const_of_diff + 1

#         diff_vals.append(const_of_diff)
        
#     max_value = max(diff_vals)
#     max_index = diff_vals.index(max_value)
#     solutions.append(max_index)
    

KeyboardInterrupt: 

In [22]:
# solutions

In [23]:
%%time

bootstrapped_sol = {}
for m in range(300):
    solutions = []
    for df in list_df:
        row_lengts = 10000
        Qjs = []
        for i in range(row_lengts):
            Qjs.append(np.random.uniform(decision_array[0], decision_array[1]))

        diff_vals = []
        for index, row in df.iterrows():
            np_array = np.array(split(row['Test'])).astype('int32')
            const_of_diff = 0
            Si = np.random.uniform(decision_array[0], decision_array[1])

            for x in range(len(np_array)):
                proba = fair_probability(Si, Qjs[x])
                response = 0
                if proba > 0.5:
                    response = 1

                # print('index,', index, 'Si', Si, 'Qj', Qjs[i], 'probability', proba, 'our_prediction', response, 'real_val', np_array[i])
                if np_array[x] != response:
                    const_of_diff = const_of_diff + 1

            diff_vals.append(const_of_diff)

        max_value = max(diff_vals)
        max_index = diff_vals.index(max_value)
        solutions.append(max_index)
        
    bootstrapped_sol[m] = solutions

CPU times: user 14h 20min 52s, sys: 32.3 s, total: 14h 21min 25s
Wall time: 14h 21min 47s


In [25]:
# with open('midput3.txt', 'w') as f:
#     f.write(json.dumps(bootstrapped_sol))

In [26]:
final_counter = {}

for j in range(50):
    final_counter[j] = []
final_counter = {}

for j in range(50):
    final_counter[j] = []
for key in bootstrapped_sol:
   for i in range(len(bootstrapped_sol[key])):
       final_counter[i].append(bootstrapped_sol[key][i])
from statistics import mode

for key in final_counter:
    out = mode(final_counter[key])
    final_counter[key] = out
output = {}
for key in final_counter:
    output[key + 1] = final_counter[key] + 1


In [27]:
with open('out_3.txt', 'w') as f:
    for key in output:
        message = 'Case #' +  str(key) + ': ' + str(output[key]) + '\n'
        f.write(message)